In [1]:
import os
import pandas as pd
import numpy as np
import difflib
import re

In [2]:
os.chdir('//to the directory shared in the Quip file')

In [3]:
os.listdir()

['XXXX Emmisions Report - no source and destination.xlsx',
 'XXXX FY21 Emissions Report with source and destination but not inbound and outbound.xlsx',
 'mapping_results-with_questions.xlsx',
 'mapping_results.xlsx',
 'mapping_results_v2.xlsx',
 'ProductCatalog.ipynb',
 'product_catalog.csv',
 'sample transportation data.xlsx']

In [4]:
ems = pd.read_excel('XXX FY21 Emissions Report with source and destination but not inbound and outbound.xlsx', skiprows = [0,1,2,3,4])

In [5]:
len(ems)

48608

In [6]:
product_catalog = pd.read_csv('product_catalog.csv')
product_catalog.columns

Index(['id', 'option_set', 'product_type', 'serialized', 'quantity_in_order',
       'reserved_in_po', 'name', 'name_short', 'unit_price', 'product_code_id',
       'order_sku', 'sku', 'status', 'msrp', 'newimage', 'main_supplier_id',
       'quantity', 'quantity_reserved', 'display_quantity', 'created',
       'modified', 'type', 'show_on_pick_ticket', 'base_manufacturer_id',
       'returnable', 'price_ruleset_id', 'vendor_id', 'online_accounting_info',
       'organizations'],
      dtype='object')

In [7]:
product_catalog = product_catalog.sort_values(['sku', 'modified']).drop_duplicates('sku', keep='last').loc[:,['name','sku','modified']]

In [8]:
len(product_catalog['name'])

3289

In [9]:
len(product_catalog['name'].unique())

2883

In [10]:
# product_catalog = product_catalog.drop_duplicates(subset =['name'])

In [11]:
ems_prod = ems.loc[:,['WHID','CommodityDescription']]

In [12]:
ems_prod['CommodityDescription'] = ems_prod['CommodityDescription'].apply(lambda x: str(x)).str.lower()
ems_prod['WHID'] = ems_prod['WHID'].apply(lambda x: str(x)).str.lower()

In [13]:
# len(ems_prod['CommodityDescription'].unique())

In [14]:
# ems_prod['CommodityDescription'].unique()

In [15]:
product_catalog['name'] = product_catalog['name'].apply(lambda x: str(x)).str.lower()
product_catalog['sku'] = product_catalog['sku'].apply(lambda x: str(x)).str.lower()

In [16]:
# WHID 

In [17]:
ems_WHID_drop_duplicates = ems_prod.drop_duplicates(subset = ['WHID'])
ems_WHID_unique = ems_WHID_drop_duplicates[~ems_WHID_drop_duplicates['WHID'].isna()]

In [18]:
ems_WHID_drop_duplicates

,WHID,CommodityDescription
0,51-004745,ral 7045 17.5 ambient dry wall
1,kswfbs9905101071j3e02hn,fbs99051010 | blackfoot15 rail set
2,fclf8522p2btl,direct to store | 1g copper optics
3,nan,console terminal server
11,23844-00-00r,mobile.charger | cord set:18awg svt 3cond 7.5ft
...,...,...
48383,n111133,fclf8522p2btl
48390,n111197,10g-sfp-t
48393,n111187,lbp-2a
48395,n111171,srt5krmxlt


In [19]:
ems_WHID_unique

,WHID,CommodityDescription
0,51-004745,ral 7045 17.5 ambient dry wall
1,kswfbs9905101071j3e02hn,fbs99051010 | blackfoot15 rail set
2,fclf8522p2btl,direct to store | 1g copper optics
3,nan,console terminal server
11,23844-00-00r,mobile.charger | cord set:18awg svt 3cond 7.5ft
...,...,...
48383,n111133,fclf8522p2btl
48390,n111197,10g-sfp-t
48393,n111187,lbp-2a
48395,n111171,srt5krmxlt


In [20]:
len(ems_WHID_unique['WHID'].unique())

1152

In [21]:
ems_WHID_unique['WHID']

0                      51-004745
1        kswfbs9905101071j3e02hn
2                  fclf8522p2btl
3                            nan
11                  23844-00-00r
                  ...           
48383                    n111133
48390                    n111197
48393                    n111187
48395                    n111171
48397                    d116296
Name: WHID, Length: 1152, dtype: object

In [22]:
ems_WHID_unique[ems_WHID_unique['WHID'] == 'a111193']

,WHID,CommodityDescription
18,a111193,a111193 - vusion 2.6 bwr eu340 f/blackgrey + b...


In [23]:
WHID_mapped_results = pd.merge(ems_WHID_unique, product_catalog, left_on = ['WHID'], right_on = ['sku'], how = 'left')
WHID_mapped_results

,WHID,CommodityDescription,name,sku,modified
0,51-004745,ral 7045 17.5 ambient dry wall,NaN,NaN,NaN
1,kswfbs9905101071j3e02hn,fbs99051010 | blackfoot15 rail set,NaN,NaN,NaN
2,fclf8522p2btl,direct to store | 1g copper optics,NaN,NaN,NaN
3,nan,console terminal server,nan,nan,2021-02-09 22:14:29
4,23844-00-00r,mobile.charger | cord set:18awg svt 3cond 7.5ft,NaN,NaN,NaN
...,...,...,...,...,...
1147,n111133,fclf8522p2btl,direct to store | 1g copper optics,n111133,2021-11-18 22:54:30
1148,n111197,10g-sfp-t,10gtek sfp+ to rj45 copper module - 10gbase-t ...,n111197,2021-11-18 22:54:44
1149,n111187,lbp-2a,lbp-2a,n111187,2021-11-18 22:54:43
1150,n111171,srt5krmxlt,ups/power supply,n111171,2021-11-18 22:54:39


In [24]:
# product_catalog[product_catalog['sku'] == 'D111992']

In [25]:
# ems_WHID_unique['trial'] = ems_WHID_unique['WHID'].map(product_catalog.set_index('sku')['name'])
# ems_WHID_unique

In [26]:
# ems_WHID_unique.loc[ems_WHID_unique['WHID'] == 'D111992',:]

In [27]:
# WHID_mapped_results.loc[WHID_mapped_results['WHID'] == 'D111992',:]

In [28]:
WHID_mapped_results_usable = WHID_mapped_results.loc[~WHID_mapped_results['sku'].isna()]
WHID_mapped_results_usable

,WHID,CommodityDescription,name,sku,modified
3,nan,console terminal server,nan,nan,2021-02-09 22:14:29
6,a111193,a111193 - vusion 2.6 bwr eu340 f/blackgrey + b...,"2.6"" (partial protective cover), black, eu - r2.0",a111193,2022-01-05 01:59:57
7,a111194,a111194,"4.2"", black, eu - r2.0",a111194,2022-01-05 01:59:57
25,n111107,acm7004-5-l | open gear cellular modem - world...,acm7004-5-l | open gear cellular modem - world...,n111107,2021-11-18 22:54:26
33,f111704,aoh ofr baffle assembly 47in & 35in,"aoh ofr baffle assembly, 47in",f111704,2022-01-24 21:16:17
...,...,...,...,...,...
1147,n111133,fclf8522p2btl,direct to store | 1g copper optics,n111133,2021-11-18 22:54:30
1148,n111197,10g-sfp-t,10gtek sfp+ to rj45 copper module - 10gbase-t ...,n111197,2021-11-18 22:54:44
1149,n111187,lbp-2a,lbp-2a,n111187,2021-11-18 22:54:43
1150,n111171,srt5krmxlt,ups/power supply,n111171,2021-11-18 22:54:39


In [29]:
results_1 = WHID_mapped_results_usable.loc[:,['WHID','CommodityDescription','sku','name']]
results_1

,WHID,CommodityDescription,sku,name
3,nan,console terminal server,nan,nan
6,a111193,a111193 - vusion 2.6 bwr eu340 f/blackgrey + b...,a111193,"2.6"" (partial protective cover), black, eu - r2.0"
7,a111194,a111194,a111194,"4.2"", black, eu - r2.0"
25,n111107,acm7004-5-l | open gear cellular modem - world...,n111107,acm7004-5-l | open gear cellular modem - world...
33,f111704,aoh ofr baffle assembly 47in & 35in,f111704,"aoh ofr baffle assembly, 47in"
...,...,...,...,...
1147,n111133,fclf8522p2btl,n111133,direct to store | 1g copper optics
1148,n111197,10g-sfp-t,n111197,10gtek sfp+ to rj45 copper module - 10gbase-t ...
1149,n111187,lbp-2a,n111187,lbp-2a
1150,n111171,srt5krmxlt,n111171,ups/power supply


In [30]:
results_1[results_1['WHID'] == 'a111193']

,WHID,CommodityDescription,sku,name
6,a111193,a111193 - vusion 2.6 bwr eu340 f/blackgrey + b...,a111193,"2.6"" (partial protective cover), black, eu - r2.0"


In [31]:
#WHID without corresponding mapped product catalog results

In [32]:
WHID_mapped_results_commodity = WHID_mapped_results.loc[WHID_mapped_results['sku'].isna()]
WHID_mapped_results_commodity

,WHID,CommodityDescription,name,sku,modified
0,51-004745,ral 7045 17.5 ambient dry wall,NaN,NaN,NaN
1,kswfbs9905101071j3e02hn,fbs99051010 | blackfoot15 rail set,NaN,NaN,NaN
2,fclf8522p2btl,direct to store | 1g copper optics,NaN,NaN,NaN
4,23844-00-00r,mobile.charger | cord set:18awg svt 3cond 7.5ft,NaN,NaN,NaN
5,c086,c086 metal sign panel mesh mounted single 18.3...,NaN,NaN,NaN
...,...,...,...,...,...
1142,f111371,"16 cup fr-fz, double sided 52-003043",NaN,NaN,NaN
1143,f111370,"8 cup fr-fz, single sided 52-003042",NaN,NaN,NaN
1144,f111235,"54"" ambient 52-003039 l6lu2058-cs-h",NaN,NaN,NaN
1145,f111234,"42"" ambient 52-003040 l6lu2059-cs-h",NaN,NaN,NaN


In [33]:
# ems_prod['combined_key'] = ems_prod['WHID'] + '_' + ems_prod['WHID']

In [34]:
# ems_prod['combined_key_index'] = ems_prod.groupby(['combined_key'], as_index = False).cumcount()+1

In [35]:
#combined_key_max = ems_prod.groupby(['combined_key'], as_index= False).agg({'combined_key_index':'max'}).\
#                                rename(columns={'combined_key_index':'combined_key_max'})

In [36]:
#ems_prod= pd.merge(ems_prod,combined_key_max,how = "left", on= ['combined_key'])

In [37]:
#ems_prod.loc[~ems_prod['WHID'].isna(),:]

In [38]:
def fuzzy_return_orginal(orginal_column, mapped_column):
    returned_list = []
    for i in mapped_column:
        try:
            returned_list.append(difflib.get_close_matches(i,orginal_column, n=1, cutoff = 0.5)[0])
        except:
            returned_list.append("")
    return returned_list

In [39]:
difflib.get_close_matches('51-005533_AOS Platform 625mmx500mm FF',['AOS | Add-on Sensor Platform 625mmx500mm FF'], n=1, cutoff = 0.5)

['AOS | Add-on Sensor Platform 625mmx500mm FF']

In [40]:
len(product_catalog['sku'].unique())

3289

In [41]:
len(product_catalog['name'].unique())

2882

In [42]:
latest_modified_product_catalog = product_catalog.sort_values(['name', 'modified']).drop_duplicates('name', keep='last').loc[:,['name','sku']]
latest_modified_product_catalog

,name,sku
3051,\tpackaging take back bin & cardboard,a111576
3131,"1 d-line cord cover black, 39 inch one-piece ...",f111767
2645,ferrari wire harness,d116265
3141,siocen【2-pack】6ft dc extension cord 5.5mm x 2...,a111609
2803,"""apple 10.2-inch ipad wi-fi + cellular - 8th ...",d116319
...,...,...
555,"| retrofit kit, rickybobby, exit",k111004
557,"| retrofit kit, rickybobby, exit, kerrigan",k111006
549,"| scr, self drilling, truss, ph, 8-16,1.0 l, ...",d111132
548,"| vhb tape, optically clear, 2 wide roll",d111131


In [43]:
commodity_word_map = fuzzy_return_orginal(latest_modified_product_catalog['name'],WHID_mapped_results_commodity['CommodityDescription'])

In [44]:
WHID_mapped_results_commodity['commodity_word_map'] = commodity_word_map
WHID_mapped_results_commodity

C:\Users\minyna\AppData\Local\Temp/ipykernel_26420/3617792973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WHID_mapped_results_commodity['commodity_word_map'] = commodity_word_map


,WHID,CommodityDescription,name,sku,modified,commodity_word_map
0,51-004745,ral 7045 17.5 ambient dry wall,NaN,NaN,NaN,ral 7045 17.5 ambient dry wall
1,kswfbs9905101071j3e02hn,fbs99051010 | blackfoot15 rail set,NaN,NaN,NaN,fbs99051010 | blackfoot15 rail set
2,fclf8522p2btl,direct to store | 1g copper optics,NaN,NaN,NaN,direct to store | 1g copper optics
4,23844-00-00r,mobile.charger | cord set:18awg svt 3cond 7.5ft,NaN,NaN,NaN,mobile.charger | cord set:18awg svt 3cond 7.5ft
5,c086,c086 metal sign panel mesh mounted single 18.3...,NaN,NaN,NaN,c086 metal sign panel mesh mounted single 18.3...
...,...,...,...,...,...,...
1142,f111371,"16 cup fr-fz, double sided 52-003043",NaN,NaN,NaN,16 cup double sided refrigeration raceway
1143,f111370,"8 cup fr-fz, single sided 52-003042",NaN,NaN,NaN,
1144,f111235,"54"" ambient 52-003039 l6lu2058-cs-h",NaN,NaN,NaN,
1145,f111234,"42"" ambient 52-003040 l6lu2059-cs-h",NaN,NaN,NaN,


In [45]:
len(commodity_word_map)

620

In [46]:
len(set(commodity_word_map))

532

In [47]:
# len(product_catalog)

In [48]:
# product_catalog.columns

In [49]:
# unique_commodity_word_map = pd.DataFrame(set(commodity_word_map), columns=['commodity_word_map'])

In [50]:
WHID_mapped_results_commodity['commodity_mapped_sku'] = WHID_mapped_results_commodity['commodity_word_map'].map(latest_modified_product_catalog.set_index('name')['sku'])
WHID_mapped_results_commodity

C:\Users\minyna\AppData\Local\Temp/ipykernel_26420/362345877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WHID_mapped_results_commodity['commodity_mapped_sku'] = WHID_mapped_results_commodity['commodity_word_map'].map(latest_modified_product_catalog.set_index('name')['sku'])


,WHID,CommodityDescription,name,sku,modified,commodity_word_map,commodity_mapped_sku
0,51-004745,ral 7045 17.5 ambient dry wall,NaN,NaN,NaN,ral 7045 17.5 ambient dry wall,f111261
1,kswfbs9905101071j3e02hn,fbs99051010 | blackfoot15 rail set,NaN,NaN,NaN,fbs99051010 | blackfoot15 rail set,n111102
2,fclf8522p2btl,direct to store | 1g copper optics,NaN,NaN,NaN,direct to store | 1g copper optics,n111133
4,23844-00-00r,mobile.charger | cord set:18awg svt 3cond 7.5ft,NaN,NaN,NaN,mobile.charger | cord set:18awg svt 3cond 7.5ft,d111060
5,c086,c086 metal sign panel mesh mounted single 18.3...,NaN,NaN,NaN,c086 metal sign panel mesh mounted single 18.3...,l111149
...,...,...,...,...,...,...,...
1142,f111371,"16 cup fr-fz, double sided 52-003043",NaN,NaN,NaN,16 cup double sided refrigeration raceway,f111371
1143,f111370,"8 cup fr-fz, single sided 52-003042",NaN,NaN,NaN,,NaN
1144,f111235,"54"" ambient 52-003039 l6lu2058-cs-h",NaN,NaN,NaN,,NaN
1145,f111234,"42"" ambient 52-003040 l6lu2059-cs-h",NaN,NaN,NaN,,NaN


In [51]:
results_2 = WHID_mapped_results_commodity.loc[WHID_mapped_results_commodity['commodity_mapped_sku'].notna(),['WHID','CommodityDescription','commodity_mapped_sku','commodity_word_map']]
results_2 = results_2.rename(columns = {'commodity_mapped_sku':'sku',
                            'commodity_word_map':'name'})
results_2

,WHID,CommodityDescription,sku,name
0,51-004745,ral 7045 17.5 ambient dry wall,f111261,ral 7045 17.5 ambient dry wall
1,kswfbs9905101071j3e02hn,fbs99051010 | blackfoot15 rail set,n111102,fbs99051010 | blackfoot15 rail set
2,fclf8522p2btl,direct to store | 1g copper optics,n111133,direct to store | 1g copper optics
4,23844-00-00r,mobile.charger | cord set:18awg svt 3cond 7.5ft,d111060,mobile.charger | cord set:18awg svt 3cond 7.5ft
5,c086,c086 metal sign panel mesh mounted single 18.3...,l111149,c086 metal sign panel mesh mounted single 18.3...
...,...,...,...,...
1139,80-000371,"number extrusion 42h, 2 sides",f111214,"number extrusion 42h, 2 sides"
1140,80-000401,"upright end trim,42h,suet style,with extended top",f111212,"upright end trim,42h,suet style,with extended ..."
1141,80-000368,"back, lower,36w,36h,painted 2 sides",f111210,"back, lower,36w,36h,painted 2 sides"
1142,f111371,"16 cup fr-fz, double sided 52-003043",f111371,16 cup double sided refrigeration raceway


In [52]:
#WHID_mapped_results_commodity['commodity_mapped_sku'] = WHID_mapped_results_commodity['commodity_word_map'].map(product_catalog.set_index('name')['sku'])
#WHID_mapped_results_commodity

In [53]:
sum(WHID_mapped_results_commodity['commodity_mapped_sku'].isna())

44

In [54]:
# WHID_mapped_results_commodity[WHID_mapped_results_commodity['commodity_mapped_sku'].isna()].head(20)

In [55]:
# WHID is null

In [56]:
# WHID_mapped_results_commodity[WHID_mapped_results_commodity['commodity_mapped_sku'].isna()]

In [57]:
#sum(ems_prod['WHID'].isna())

In [58]:
# ems_prod

In [59]:
ems_prod_WHID_null = ems_prod[ems_prod['WHID'] == 'nan'].drop_duplicates(subset = ['CommodityDescription'])
ems_prod_WHID_null

,WHID,CommodityDescription
3,nan,console terminal server
4,nan,ftlx8574d3bcl
5,nan,"2.6"" freezer (with partial protective cover),"
6,nan,"1.6"" black, us - r2.0"
7,nan,"2.2“ black, us - r2.0"
...,...,...
48536,nan,device id: 9908468426
48537,nan,device id: 9908463787
48538,nan,device id: 9908465654
48539,nan,device id: 9908465355


In [60]:
# WHID_mapped_results_commodity

In [61]:
WHID_null_map = fuzzy_return_orginal(latest_modified_product_catalog['name'],ems_prod_WHID_null['CommodityDescription'])

In [62]:
ems_prod_WHID_null['WHID_null_map'] = WHID_null_map
ems_prod_WHID_null

,WHID,CommodityDescription,WHID_null_map
3,nan,console terminal server,console terminal server
4,nan,ftlx8574d3bcl,ftlx8574d3bcl
5,nan,"2.6"" freezer (with partial protective cover),","2.6"" freezer (with partial protective cover), ..."
6,nan,"1.6"" black, us - r2.0","1.6"" black, us - r2.0"
7,nan,"2.2“ black, us - r2.0","2.2“ black, us - r2.0"
...,...,...,...
48536,nan,device id: 9908468426,
48537,nan,device id: 9908463787,
48538,nan,device id: 9908465654,
48539,nan,device id: 9908465355,


In [63]:
ems_prod_WHID_null['WHID_null_map_sku'] = ems_prod_WHID_null['WHID_null_map'].map(latest_modified_product_catalog.set_index('name')['sku'])
ems_prod_WHID_null

,WHID,CommodityDescription,WHID_null_map,WHID_null_map_sku
3,nan,console terminal server,console terminal server,n111225
4,nan,ftlx8574d3bcl,ftlx8574d3bcl,n111186
5,nan,"2.6"" freezer (with partial protective cover),","2.6"" freezer (with partial protective cover), ...",a111095
6,nan,"1.6"" black, us - r2.0","1.6"" black, us - r2.0",a111183
7,nan,"2.2“ black, us - r2.0","2.2“ black, us - r2.0",a111184
...,...,...,...,...
48536,nan,device id: 9908468426,,NaN
48537,nan,device id: 9908463787,,NaN
48538,nan,device id: 9908465654,,NaN
48539,nan,device id: 9908465355,,NaN


In [64]:
sum(ems_prod_WHID_null['WHID_null_map_sku'].isnull())

283

In [65]:
sum(~ems_prod_WHID_null['WHID_null_map_sku'].isnull())

1022

In [66]:
results_3 = ems_prod_WHID_null.loc[ems_prod_WHID_null['WHID_null_map_sku'].notna(),\
                                   ['WHID','CommodityDescription','WHID_null_map_sku','WHID_null_map']]
results_3 = results_3.rename(columns = {'WHID_null_map_sku':'sku',\
                             'WHID_null_map':'name'})
results_3

,WHID,CommodityDescription,sku,name
3,nan,console terminal server,n111225,console terminal server
4,nan,ftlx8574d3bcl,n111186,ftlx8574d3bcl
5,nan,"2.6"" freezer (with partial protective cover),",a111095,"2.6"" freezer (with partial protective cover), ..."
6,nan,"1.6"" black, us - r2.0",a111183,"1.6"" black, us - r2.0"
7,nan,"2.2“ black, us - r2.0",a111184,"2.2“ black, us - r2.0"
...,...,...,...,...
47780,nan,dash cart ab62,e000001,dash cart
47844,nan,50-006405 & 35” baffle assemblies,f111704,"aoh ofr baffle assembly, 47in"
48517,nan,hs3100 single battery,d116276,hs3100 single battery
48519,nan,earphone covers,a111363,earphone covers


In [67]:
# no mapping results

In [68]:
WHID_mapped_results_commodity.columns

Index(['WHID', 'CommodityDescription', 'name', 'sku', 'modified',
       'commodity_word_map', 'commodity_mapped_sku'],
      dtype='object')

In [69]:
ems_prod_WHID_null.columns

Index(['WHID', 'CommodityDescription', 'WHID_null_map', 'WHID_null_map_sku'], dtype='object')

In [70]:
no_match = pd.concat([WHID_mapped_results_commodity.loc[WHID_mapped_results_commodity['commodity_mapped_sku'].isna(),['WHID','CommodityDescription','commodity_word_map','commodity_mapped_sku']]\
                                        .rename(columns = {'WHID':'WHID',\
                                                'CommodityDescription':'CommodityDescription',\
                                                'commodity_word_map':'mapped_name',\
                                                'commodity_mapped_sku' : 'mapped_sku'})
                                                ,ems_prod_WHID_null.loc[ems_prod_WHID_null['WHID_null_map_sku'].isnull(),:].rename(columns = {'WHID':'WHID',\
                                                'CommodityDescription':'CommodityDescription',\
                                                'WHID_null_map':'mapped_name',\
                                                'WHID_null_map_sku' : 'mapped_sku'})], ignore_index=True)

In [71]:
no_match

,WHID,CommodityDescription,mapped_name,mapped_sku
0,f1111849,f111184,,NaN
1,b01hu5sfp4,drying mat for 3-comp sink- 1'x1' tile (6/,,NaN
2,51-003538,nuc,,NaN
3,2d-06177241,f111198,,NaN
4,d1112054,51-004964-192units,,NaN
...,...,...,...,...
322,nan,device id: 9908468426,,NaN
323,nan,device id: 9908463787,,NaN
324,nan,device id: 9908465654,,NaN
325,nan,device id: 9908465355,,NaN


In [72]:
len(no_match['CommodityDescription'].unique())

317

In [73]:
no_match['WHID'][no_match['WHID']!='nan'] = no_match['WHID'][no_match['WHID']!='nan'].apply(lambda x: str(x)).str.lower()
no_match

,WHID,CommodityDescription,mapped_name,mapped_sku
0,f1111849,f111184,,NaN
1,b01hu5sfp4,drying mat for 3-comp sink- 1'x1' tile (6/,,NaN
2,51-003538,nuc,,NaN
3,2d-06177241,f111198,,NaN
4,d1112054,51-004964-192units,,NaN
...,...,...,...,...
322,nan,device id: 9908468426,,NaN
323,nan,device id: 9908463787,,NaN
324,nan,device id: 9908465654,,NaN
325,nan,device id: 9908465355,,NaN


In [74]:
no_match_WHID_reshape = no_match.loc[no_match['WHID']!='nan',:]
no_match_WHID_reshape

,WHID,CommodityDescription,mapped_name,mapped_sku
0,f1111849,f111184,,NaN
1,b01hu5sfp4,drying mat for 3-comp sink- 1'x1' tile (6/,,NaN
2,51-003538,nuc,,NaN
3,2d-06177241,f111198,,NaN
4,d1112054,51-004964-192units,,NaN
5,5712-54x-o-ac-f-us,5712-54x-o-ac-f-us | as5712-54x 48-port 10g s...,,NaN
6,d19c118,dac4111.qt8,,NaN
7,d111258,d111258,,NaN
8,a111307,a111307,,NaN
9,d116308,d116308,,NaN


In [75]:
no_match_WHID_reshape['WHID_reshape'] = no_match_WHID_reshape['WHID'].str.replace("/",",")

C:\Users\minyna\AppData\Local\Temp/ipykernel_26420/2380794888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_match_WHID_reshape['WHID_reshape'] = no_match_WHID_reshape['WHID'].str.replace("/",",")


In [76]:
no_match_WHID_reshape_stack = no_match_WHID_reshape.set_index(['CommodityDescription','WHID'])['WHID_reshape'].str.split(",",expand = True).stack().reset_index(level =2, drop = True).reset_index(name = 'WHID_reshape')
no_match_WHID_reshape_stack

,CommodityDescription,WHID,WHID_reshape
0,f111184,f1111849,f1111849
1,drying mat for 3-comp sink- 1'x1' tile (6/,b01hu5sfp4,b01hu5sfp4
2,nuc,51-003538,51-003538
3,f111198,2d-06177241,2d-06177241
4,51-004964-192units,d1112054,d1112054
5,5712-54x-o-ac-f-us | as5712-54x 48-port 10g s...,5712-54x-o-ac-f-us,5712-54x-o-ac-f-us
6,dac4111.qt8,d19c118,d19c118
7,d111258,d111258,d111258
8,a111307,a111307,a111307
9,d116308,d116308,d116308


In [77]:
product_catalog['smaller_case_sku'] = product_catalog['sku'].apply(lambda x: str(x)).str.lower()

In [78]:
no_match_WHID_reshape_stack['mapped_name'] = no_match_WHID_reshape_stack['WHID_reshape'].map(product_catalog.set_index('smaller_case_sku')['name'])
no_match_WHID_reshape_stack

,CommodityDescription,WHID,WHID_reshape,mapped_name
0,f111184,f1111849,f1111849,NaN
1,drying mat for 3-comp sink- 1'x1' tile (6/,b01hu5sfp4,b01hu5sfp4,NaN
2,nuc,51-003538,51-003538,NaN
3,f111198,2d-06177241,2d-06177241,NaN
4,51-004964-192units,d1112054,d1112054,NaN
5,5712-54x-o-ac-f-us | as5712-54x 48-port 10g s...,5712-54x-o-ac-f-us,5712-54x-o-ac-f-us,NaN
6,dac4111.qt8,d19c118,d19c118,NaN
7,d111258,d111258,d111258,NaN
8,a111307,a111307,a111307,NaN
9,d116308,d116308,d116308,NaN


In [79]:
sum(no_match_WHID_reshape_stack['mapped_name'].notna())

21

In [80]:
results_4 = no_match_WHID_reshape_stack.loc[no_match_WHID_reshape_stack['mapped_name'].notna(),:]
results_4['sku'] = results_4['WHID_reshape']
results_4 = results_4.loc[:,['WHID','CommodityDescription','sku','mapped_name']].rename(columns = {'mapped_name':'name'})
results_4

C:\Users\minyna\AppData\Local\Temp/ipykernel_26420/3651746985.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_4['sku'] = results_4['WHID_reshape']


,WHID,CommodityDescription,sku,name
10,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111236,hl oprs small back support
11,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111237,hl opss ridge track for back support
12,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111238,hl osp 60mm pusher add on
13,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111239,hl osp 80mm pusher add on
14,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111248,hl clear riser for ofr 605mmw 60mmh
15,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111249,hl clear riser for ofr 895mmw 60mmh
16,l111308/l111309,e12120/e20122,l111308,12oz double wall paper cup generic black
17,l111308/l111309,e12120/e20122,l111309,10-20oz lid for paper cup white matt
19,l111299/l111300,16434-46-0/17819-46-0,l111299,"2 col x 8 row, 1.5 l/r, .5 t/b - 65# astrobrig..."
20,l111299/l111300,16434-46-0/17819-46-0,l111300,"4 cols x 8 rows - .5"" t/b margins and .25"" l/r..."


In [81]:
# no match no whid

In [82]:
no_match_no_WHID = no_match.loc[no_match['WHID'] =='nan',:]
no_match_no_WHID

,WHID,CommodityDescription,mapped_name,mapped_sku
44,nan,(aos) platforms,,NaN
45,nan,acctim delta radio controlled digital clock si...,,NaN
46,nan,default,,NaN
47,nan,co2 tank wrench | kegworks co2 air tank wrench,,NaN
48,nan,uv flashlight | uv flashlight black light 365n...,,NaN
...,...,...,...,...
322,nan,device id: 9908468426,,NaN
323,nan,device id: 9908463787,,NaN
324,nan,device id: 9908465654,,NaN
325,nan,device id: 9908465355,,NaN


In [83]:
no_match_no_WHID_filter = no_match_no_WHID.loc[no_match_no_WHID['CommodityDescription'].str.contains(r"^[a-z]\d{6}$"),:]
no_match_no_WHID_filter

,WHID,CommodityDescription,mapped_name,mapped_sku
53,nan,n111205,,NaN
69,nan,a111226,,NaN
73,nan,n111191,,NaN
119,nan,f111178,,NaN
127,nan,d112076,,NaN
133,nan,d112242,,NaN
136,nan,d111991,,NaN
145,nan,f111301,,NaN
256,nan,a111133,,NaN
281,nan,d112216,,NaN


In [84]:
no_match_no_WHID_filter['mapped_sku_smaller_case'] = no_match_no_WHID_filter['CommodityDescription'].map(product_catalog.set_index('smaller_case_sku')['name'])
no_match_no_WHID_filter

C:\Users\minyna\AppData\Local\Temp/ipykernel_26420/412047413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_match_no_WHID_filter['mapped_sku_smaller_case'] = no_match_no_WHID_filter['CommodityDescription'].map(product_catalog.set_index('smaller_case_sku')['name'])


,WHID,CommodityDescription,mapped_name,mapped_sku,mapped_sku_smaller_case
53,nan,n111205,,NaN,4u 15kva ups (eu version)
69,nan,a111226,,NaN,"esl 2.6"" snapon (each pack contains 50 pcs)"
73,nan,n111191,,NaN,"220v mdf, non-locking, right"
119,nan,f111178,,NaN,"upright,90h,extra slot for deck"
127,nan,d112076,,NaN,ferrari assembly mp
133,nan,d112242,,NaN,aoh ofr baffle left sleeve bracket
136,nan,d111991,,NaN,speaker
145,nan,f111301,,NaN,aos platform (mp) | 48x13ab | dark matte gra...
256,nan,a111133,,NaN,esl fixture for arneg refrigerator (ofr) shelf
281,nan,d112216,,NaN,aoh 17.5in slat wall


In [85]:
no_match_no_WHID_filter['sku'] = no_match_no_WHID_filter['CommodityDescription']
no_match_no_WHID_filter

C:\Users\minyna\AppData\Local\Temp/ipykernel_26420/677516801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_match_no_WHID_filter['sku'] = no_match_no_WHID_filter['CommodityDescription']


,WHID,CommodityDescription,mapped_name,mapped_sku,mapped_sku_smaller_case,sku
53,nan,n111205,,NaN,4u 15kva ups (eu version),n111205
69,nan,a111226,,NaN,"esl 2.6"" snapon (each pack contains 50 pcs)",a111226
73,nan,n111191,,NaN,"220v mdf, non-locking, right",n111191
119,nan,f111178,,NaN,"upright,90h,extra slot for deck",f111178
127,nan,d112076,,NaN,ferrari assembly mp,d112076
133,nan,d112242,,NaN,aoh ofr baffle left sleeve bracket,d112242
136,nan,d111991,,NaN,speaker,d111991
145,nan,f111301,,NaN,aos platform (mp) | 48x13ab | dark matte gra...,f111301
256,nan,a111133,,NaN,esl fixture for arneg refrigerator (ofr) shelf,a111133
281,nan,d112216,,NaN,aoh 17.5in slat wall,d112216


In [86]:
results_5 = no_match_no_WHID_filter.loc[:,['WHID','CommodityDescription','sku','mapped_sku_smaller_case']].rename(columns = \
                                                        {'mapped_sku_smaller_case':'name'})
results_5

,WHID,CommodityDescription,sku,name
53,nan,n111205,n111205,4u 15kva ups (eu version)
69,nan,a111226,a111226,"esl 2.6"" snapon (each pack contains 50 pcs)"
73,nan,n111191,n111191,"220v mdf, non-locking, right"
119,nan,f111178,f111178,"upright,90h,extra slot for deck"
127,nan,d112076,d112076,ferrari assembly mp
133,nan,d112242,d112242,aoh ofr baffle left sleeve bracket
136,nan,d111991,d111991,speaker
145,nan,f111301,f111301,aos platform (mp) | 48x13ab | dark matte gra...
256,nan,a111133,a111133,esl fixture for arneg refrigerator (ofr) shelf
281,nan,d112216,d112216,aoh 17.5in slat wall


In [87]:
results_1.head()

,WHID,CommodityDescription,sku,name
3,nan,console terminal server,nan,nan
6,a111193,a111193 - vusion 2.6 bwr eu340 f/blackgrey + b...,a111193,"2.6"" (partial protective cover), black, eu - r2.0"
7,a111194,a111194,a111194,"4.2"", black, eu - r2.0"
25,n111107,acm7004-5-l | open gear cellular modem - world...,n111107,acm7004-5-l | open gear cellular modem - world...
33,f111704,aoh ofr baffle assembly 47in & 35in,f111704,"aoh ofr baffle assembly, 47in"


In [88]:
results_2.head()

,WHID,CommodityDescription,sku,name
0,51-004745,ral 7045 17.5 ambient dry wall,f111261,ral 7045 17.5 ambient dry wall
1,kswfbs9905101071j3e02hn,fbs99051010 | blackfoot15 rail set,n111102,fbs99051010 | blackfoot15 rail set
2,fclf8522p2btl,direct to store | 1g copper optics,n111133,direct to store | 1g copper optics
4,23844-00-00r,mobile.charger | cord set:18awg svt 3cond 7.5ft,d111060,mobile.charger | cord set:18awg svt 3cond 7.5ft
5,c086,c086 metal sign panel mesh mounted single 18.3...,l111149,c086 metal sign panel mesh mounted single 18.3...


In [89]:
results_3.head()

,WHID,CommodityDescription,sku,name
3,nan,console terminal server,n111225,console terminal server
4,nan,ftlx8574d3bcl,n111186,ftlx8574d3bcl
5,nan,"2.6"" freezer (with partial protective cover),",a111095,"2.6"" freezer (with partial protective cover), ..."
6,nan,"1.6"" black, us - r2.0",a111183,"1.6"" black, us - r2.0"
7,nan,"2.2“ black, us - r2.0",a111184,"2.2“ black, us - r2.0"


In [90]:
results_4.head()

,WHID,CommodityDescription,sku,name
10,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111236,hl oprs small back support
11,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111237,hl opss ridge track for back support
12,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111238,hl osp 60mm pusher add on
13,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111239,hl osp 80mm pusher add on
14,a111236/a111237/a111238/a111239/a111248/a111249,387119/303295/302399/857441/154352/154354,a111248,hl clear riser for ofr 605mmw 60mmh


In [91]:
results_5.head()

,WHID,CommodityDescription,sku,name
53,nan,n111205,n111205,4u 15kva ups (eu version)
69,nan,a111226,a111226,"esl 2.6"" snapon (each pack contains 50 pcs)"
73,nan,n111191,n111191,"220v mdf, non-locking, right"
119,nan,f111178,f111178,"upright,90h,extra slot for deck"
127,nan,d112076,d112076,ferrari assembly mp


In [92]:
final_results = results_1.append(results_2, ignore_index = True)
final_results = final_results.append(results_3, ignore_index = True)
final_results = final_results.append(results_4, ignore_index = True)
final_results = final_results.append(results_5, ignore_index = True)
final_results

,WHID,CommodityDescription,sku,name
0,nan,console terminal server,nan,nan
1,a111193,a111193 - vusion 2.6 bwr eu340 f/blackgrey + b...,a111193,"2.6"" (partial protective cover), black, eu - r2.0"
2,a111194,a111194,a111194,"4.2"", black, eu - r2.0"
3,n111107,acm7004-5-l | open gear cellular modem - world...,n111107,acm7004-5-l | open gear cellular modem - world...
4,f111704,aoh ofr baffle assembly 47in & 35in,f111704,"aoh ofr baffle assembly, 47in"
...,...,...,...,...
2159,nan,a111133,a111133,esl fixture for arneg refrigerator (ofr) shelf
2160,nan,d112216,d112216,aoh 17.5in slat wall
2161,nan,d111953,d111953,aoh | 1250mm gondola ff
2162,nan,a111310,a111310,aoh | ral7045 35in extended bracket


In [93]:
final_results['sku'] = final_results['sku'].str.lower()
final_results['WHID']  = final_results['WHID'].str.lower()
final_results

,WHID,CommodityDescription,sku,name
0,nan,console terminal server,nan,nan
1,a111193,a111193 - vusion 2.6 bwr eu340 f/blackgrey + b...,a111193,"2.6"" (partial protective cover), black, eu - r2.0"
2,a111194,a111194,a111194,"4.2"", black, eu - r2.0"
3,n111107,acm7004-5-l | open gear cellular modem - world...,n111107,acm7004-5-l | open gear cellular modem - world...
4,f111704,aoh ofr baffle assembly 47in & 35in,f111704,"aoh ofr baffle assembly, 47in"
...,...,...,...,...
2159,nan,a111133,a111133,esl fixture for arneg refrigerator (ofr) shelf
2160,nan,d112216,d112216,aoh 17.5in slat wall
2161,nan,d111953,d111953,aoh | 1250mm gondola ff
2162,nan,a111310,a111310,aoh | ral7045 35in extended bracket


In [94]:
final_results = final_results.loc[final_results['sku']!= 'nan',:]

In [95]:
final_results.to_csv('mapping.csv')

In [96]:
#final_results.drop_duplicates(subset=['WHID'])

In [97]:
# final mapping

In [98]:
ems.head()

,LoadNum,ReferenceNum,Inner Pack Quantity,ActualPallets,ActualWeight,Mode,Miles,WHID,CommodityDescription,OriginName,OriginCity,OriginState,OriginPostalCode,DestinationName,DestinationCity,DestinationState,DestinationPostalCode,CarrierName,Business Unit
0,370957008.0,AMZN-5008189-DI-7471-18,1.0,0.03,11.00,Parcel,1114.0,51-004745,RAL 7045 17.5 ambient dry wall,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,SHJ,Los Angeles,CA,91423,UPS Ground - U03,TMC - Amazon Go
1,370957101.0,AMZN-5008946-DI-8214-1,2.0,0.01,209.85,LTL,1150.0,KSWFBS9905101071J3E02HN,FBS99051010 | Blackfoot15 Rail set,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go
2,370957101.0,AMZN-5008946-DI-8214-1,1.0,NaN,209.85,LTL,1150.0,FCLF8522P2BTL,Direct To Store | 1G Copper Optics,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go
3,370957101.0,AMZN-5008946-DI-8214-1,3.0,0.03,209.85,LTL,1150.0,NaN,Console Terminal Server,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go
4,370957101.0,AMZN-5008946-DI-8214-1,332.0,0.96,209.85,LTL,1150.0,NaN,FTLX8574D3BCL,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go


In [99]:
len(ems)

48608

In [100]:
ems

,LoadNum,ReferenceNum,Inner Pack Quantity,ActualPallets,ActualWeight,Mode,Miles,WHID,CommodityDescription,OriginName,OriginCity,OriginState,OriginPostalCode,DestinationName,DestinationCity,DestinationState,DestinationPostalCode,CarrierName,Business Unit
0,370957008.0,AMZN-5008189-DI-7471-18,1.0,0.03,11.00,Parcel,1114.0,51-004745,RAL 7045 17.5 ambient dry wall,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,SHJ,Los Angeles,CA,91423,UPS Ground - U03,TMC - Amazon Go
1,370957101.0,AMZN-5008946-DI-8214-1,2.0,0.01,209.85,LTL,1150.0,KSWFBS9905101071J3E02HN,FBS99051010 | Blackfoot15 Rail set,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go
2,370957101.0,AMZN-5008946-DI-8214-1,1.0,NaN,209.85,LTL,1150.0,FCLF8522P2BTL,Direct To Store | 1G Copper Optics,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go
3,370957101.0,AMZN-5008946-DI-8214-1,3.0,0.03,209.85,LTL,1150.0,NaN,Console Terminal Server,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go
4,370957101.0,AMZN-5008946-DI-8214-1,332.0,0.96,209.85,LTL,1150.0,NaN,FTLX8574D3BCL,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48603,370952828.0,AMZN-5007143-DI-4765-2,2.0,0.05,11.40,Parcel,2845.0,51-004972,Catalina DV,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,MAF6,Bensalem,PA,19020,UPS Next Day Air Saver - U13,TMC - Amazon Go
48604,370954658.0,AMZN-5009384-DI-8624-1,1.0,0.01,9.70,Parcel,23.0,C086,C086 METAL SIGN PANEL MESH MOUNTED SINGLE 18.3...,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,IBF3,Seattle,WA,98101,UPS Ground - U03,TMC - Amazon Go
48605,370956960.0,AMZN-5009370-DI-8610-1,1.0,0.01,0.71,Parcel,2048.0,GCFAZBBQ03-MPR,GCFAZBBQ03-MPR | 3M Multimode Fiber patch cabl...,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,JOR2,Oak Brook,IL,60523,UPS Next Day Air - U01,Priority 3 Expedite - Amazon Go - TMC
48606,370956960.0,AMZN-5009370-DI-8610-1,1.0,0.01,0.71,Parcel,2048.0,NaN,FTLX8574D3BCL,Amazon GSE1 c/o Crane Worldwide,Kent,WA,98032,JOR2,Oak Brook,IL,60523,UPS Next Day Air - U01,Priority 3 Expedite - Amazon Go - TMC


In [101]:
ems = ems.dropna(subset = ['LoadNum'])

In [102]:
len(ems)

44468

In [103]:
ems['WHID'] = ems['WHID'].apply(lambda x: str(x)).str.lower()
ems['CommodityDescription'] = ems['CommodityDescription'].apply(lambda x: str(x)).str.lower()

C:\Users\minyna\AppData\Local\Temp/ipykernel_26420/4148695745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ems['WHID'] = ems['WHID'].apply(lambda x: str(x)).str.lower()
C:\Users\minyna\AppData\Local\Temp/ipykernel_26420/4148695745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ems['CommodityDescription'] = ems['CommodityDescription'].apply(lambda x: str(x)).str.lower()


In [104]:
#join for WHID 

In [105]:
# merge for WHID

In [106]:
merged_results = ems.merge(final_results.loc[:,['WHID','CommodityDescription','sku','name']], on = ['WHID','CommodityDescription'], how = 'left')

In [107]:
merged_results

,LoadNum,ReferenceNum,Inner Pack Quantity,ActualPallets,ActualWeight,Mode,Miles,WHID,CommodityDescription,OriginName,...,OriginState,OriginPostalCode,DestinationName,DestinationCity,DestinationState,DestinationPostalCode,CarrierName,Business Unit,sku,name
0,370957008.0,AMZN-5008189-DI-7471-18,1.0,0.03,11.00,Parcel,1114.0,51-004745,ral 7045 17.5 ambient dry wall,Amazon GSE1 c/o Crane Worldwide,...,WA,98032,SHJ,Los Angeles,CA,91423,UPS Ground - U03,TMC - Amazon Go,f111261,ral 7045 17.5 ambient dry wall
1,370957101.0,AMZN-5008946-DI-8214-1,2.0,0.01,209.85,LTL,1150.0,kswfbs9905101071j3e02hn,fbs99051010 | blackfoot15 rail set,Amazon GSE1 c/o Crane Worldwide,...,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go,n111102,fbs99051010 | blackfoot15 rail set
2,370957101.0,AMZN-5008946-DI-8214-1,1.0,NaN,209.85,LTL,1150.0,fclf8522p2btl,direct to store | 1g copper optics,Amazon GSE1 c/o Crane Worldwide,...,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go,n111133,direct to store | 1g copper optics
3,370957101.0,AMZN-5008946-DI-8214-1,3.0,0.03,209.85,LTL,1150.0,nan,console terminal server,Amazon GSE1 c/o Crane Worldwide,...,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go,n111225,console terminal server
4,370957101.0,AMZN-5008946-DI-8214-1,332.0,0.96,209.85,LTL,1150.0,nan,ftlx8574d3bcl,Amazon GSE1 c/o Crane Worldwide,...,WA,98032,MAC3,La Verne,CA,91750,UPS Ground - U03,TMC - Amazon Go,n111186,ftlx8574d3bcl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44478,370952828.0,AMZN-5007143-DI-4765-2,2.0,0.05,11.40,Parcel,2845.0,51-004972,catalina dv,Amazon GSE1 c/o Crane Worldwide,...,WA,98032,MAF6,Bensalem,PA,19020,UPS Next Day Air Saver - U13,TMC - Amazon Go,f111288,catalina dv
44479,370954658.0,AMZN-5009384-DI-8624-1,1.0,0.01,9.70,Parcel,23.0,c086,c086 metal sign panel mesh mounted single 18.3...,Amazon GSE1 c/o Crane Worldwide,...,WA,98032,IBF3,Seattle,WA,98101,UPS Ground - U03,TMC - Amazon Go,l111149,c086 metal sign panel mesh mounted single 18.3...
44480,370956960.0,AMZN-5009370-DI-8610-1,1.0,0.01,0.71,Parcel,2048.0,gcfazbbq03-mpr,gcfazbbq03-mpr | 3m multimode fiber patch cabl...,Amazon GSE1 c/o Crane Worldwide,...,WA,98032,JOR2,Oak Brook,IL,60523,UPS Next Day Air - U01,Priority 3 Expedite - Amazon Go - TMC,n111126,gcfazbbq03-mpr | 3m multimode fiber patch cabl...
44481,370956960.0,AMZN-5009370-DI-8610-1,1.0,0.01,0.71,Parcel,2048.0,nan,ftlx8574d3bcl,Amazon GSE1 c/o Crane Worldwide,...,WA,98032,JOR2,Oak Brook,IL,60523,UPS Next Day Air - U01,Priority 3 Expedite - Amazon Go - TMC,n111186,ftlx8574d3bcl


In [108]:
merged_results.to_excel('mapping_results_v2.xlsx')